In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import sqlite3
import seaborn as sns
import patsy
from sklearn.decomposition import PCA
from lifelines import KaplanMeierFitter
from matplotlib.ticker import StrMethodFormatter
from statsmodels.stats.multitest import fdrcorrection

# import rpy2.ipython
# %load_ext rpy2.ipython.rmagic

from scripts.lib.stats import raise_low, lrt_phreg, phreg_aic
from scripts.lib.plotting import boxplot_with_points, load_style

from contextlib import contextmanager

def model_to_eval_input(model, drop=None, include=None):
    all_vars = [v.name for v in model.vars]
    if include is None:
        if drop is None:
            include = all_vars
        else:
            include = set(all_vars) - set(drop)
    else:
        if drop is not None:
            include = set(include) - set(drop)
    out = {getattr(model, key): model.test_point[key]
            for key in model.test_point
            if key in include}
    return out


@contextmanager
def temp_assign(update_dict):
    saved_dict = {}
    for var in update_dict:
        saved_dict[var] = var.get_value()
        var.set_value(update_dict[var])
    yield
    for var in update_dict:
        var.set_value(saved_dict[var])
        
def sample_ppc(model, trace, update_shared_dict={}, samples=1, **kwargs):
    with model:
        with temp_assign(update_shared_dict):
            sim = pm.sample_ppc(trace, samples=samples, **kwargs)
            return sim
        
def age_at_first(events, interval_bounds=None):
    events = np.asanyarray(events)
    if interval_bounds is None:
        interval_bounds = np.arange(len(events))
    return interval_bounds[np.argmax(events > 0, axis=-1)]

In [ ]:
loaded_style = load_style('paper')

color_map = loaded_style['color_map']
mark_map = loaded_style['mark_map']
assign_significance_symbol = loaded_style['assign_significance_symbol']
figure_ft = 'png'

In [ ]:
from scripts.lib.data import load_data
loaded_data = load_data('res/C2013.results.db')

In [ ]:
mouse = loaded_data['mouse']
conc = loaded_data['conc']
abund = loaded_data['abund']

In [ ]:
import pymc3 as pm
import theano
import theano.tensor as tt

def exposure_m(exit, intervals, entry=None):
    """Build per-subject exposure matrix.
    
    This matrix has entries in cells corresonding to
    the amount of time spent in each time interval.
    
    """
    if entry is None:
        entry = np.zeros_like(exit)
        
    assert entry.min() >= intervals[0]
    assert exit.max() <= intervals[-1]
    assert np.all(entry < exit)
    interval_size = intervals[1:] - intervals[:-1]
    assert np.all(interval_size > 0)
    
    
    after_entry = np.less.outer(entry, intervals[1:])
    before_exit = np.greater.outer(exit, intervals[:-1])
    
    idx_first_in = (~after_entry).sum(axis=1)
    idx_last_in = before_exit.sum(axis=1) - 1
    time_present_first = intervals[idx_first_in + 1] - entry
    time_present_last = exit - intervals[idx_last_in]
    
    exposed = (after_entry & before_exit).astype(float) * interval_size
    exposed[np.arange(len(entry)), idx_first_in] = time_present_first
    exposed[np.arange(len(entry)), idx_last_in] = time_present_last
    
    return exposed

def test_exposure_m():
    entry = np.array([0.9, 1.0, 1.1, 0.9, 1.0, 1.1, 0.9, 1.0, 1.1])
    exit = np.array([1.9, 1.9, 1.9, 2, 2, 2, 2.1, 2.1, 2.1])
    intervals = np.arange(0, 4)
    out = exposure_m(exit, intervals, entry=entry)
    assert np.allclose(out,
                       np.array([[ 0.1,  0.9,  0. ],
                                 [ 0. ,  0.9,  0. ],
                                 [ 0. ,  0.9,  0. ],
                                 [ 0.1,  1. ,  0. ],
                                 [ 0. ,  1. ,  0. ],
                                 [ 0. ,  1. ,  0. ],
                                 [ 0.1,  1. ,  0.1],
                                 [ 0. ,  1. ,  0.1],
                                 [ 0. ,  0.9,  0.1]])
                      ), out

    
def event_m(exit, intervals, censored=None):
    """Build per-subject event matrix.
    
    This matrix has entries in cells corresponding to the
    time intervals during which a subject died.
    
    Events which occur at the interval break are included
    in the following interval.
    
    """
    interval_size = intervals[1:] - intervals[:-1]
    assert np.all(interval_size > 0)
    assert exit.min() >= intervals[0]
    assert exit.max() <= intervals[-1]
    after_exit_interval = np.less_equal.outer(exit, intervals[:-1])
    before_exit_interval = np.greater.outer(exit, intervals[1:])
#    print(after_exit_interval)
#    print(before_exit_interval)
    event = (~after_exit_interval & ~before_exit_interval).astype(int)
    if censored is not None:
        event[np.argwhere(censored),:] = 0
#    print(event)
    return event

def test_event_m():
    exit = np.array([1.9, 2, 2.1])
    intervals = np.arange(0, 4)
    out = event_m(exit, intervals)
    assert np.all(out == np.array([[0, 1, 0],
                                   [0, 1, 0],
                                   [0, 0, 1]]))
    
test_event_m()
test_exposure_m()

In [ ]:
conc_adj = conc.copy()
conc_adj.butyrate = raise_low(conc_adj.butyrate)

data = (mouse[mouse.cohort.isin(['C2013'])]
             .join(conc_adj).join(abund)
             .dropna(subset=['age_at_death_or_censor', 'age_at_collect',
                             'propionate', 'Otu0001'])
        )
data['censored'] = ~data.dead.astype(bool)

njects = len(data)

# Discretization
interval_length = 10
interval_bounds = np.arange(data.age_at_collect.min(),
                            data.age_at_death_or_censor.max() + interval_length + 1,
                            interval_length)
m_intervals = len(interval_bounds) - 1

# Design
x_di = patsy.dmatrix('treatment * site * sex', data=data).design_info
u = len(x_di.column_names) - 1
x_design = lambda d: (patsy.build_design_matrices([x_di], d, return_type='dataframe')[0]
                           .drop('Intercept', axis='columns'))

# SCFAs
y_di = patsy.dmatrix('propionate + butyrate + acetate', data=data).design_info
v = len(y_di.column_names) - 1
y_design = lambda d: (patsy.build_design_matrices([y_di], d,return_type='dataframe')[0]
                           .drop('Intercept', axis='columns'))

In [ ]:
with pm.Model() as modelA:
    x = theano.shared(x_design(data).values)
    y = theano.shared(y_design(data).values)
    exposure = theano.shared(exposure_m(exit=data.age_at_death_or_censor,
                                        intervals=interval_bounds,
                                        entry=data.age_at_collect))
    event = theano.shared(event_m(exit=data.age_at_death_or_censor,
                                  intervals=interval_bounds,
                                  censored=data.censored))
    
    # SCFA model
    alpha = pm.Normal('alpha', sd=10, shape=(u, v))
    alpha_0 = pm.Normal('alpha_0', sd=10, shape=v)
    # TODO: Rename expect_y to expect_y_log
    expect_y = pm.Deterministic('expect_y', tt.dot(x, alpha) + alpha_0)
    chol_packed = pm.LKJCholeskyCov('chol_packed', n=v, eta=2, sd_dist=pm.HalfCauchy.dist(beta=2.5, shape=v))
    chol = pm.Deterministic('chol', pm.expand_packed_triangular(3, chol_packed))
    obs_y = pm.MvNormal('obs_y', mu=expect_y, chol=chol, observed=tt.log(y))

    lambda0 = pm.Gamma('lambda0', 0.01, 0.01, shape=m_intervals, testval=np.repeat(0.01, m_intervals))
    beta_x = pm.Normal('beta_x', sd=10, shape=(u, 1))
    beta_y = pm.Normal('beta_y', sd=10, shape=(v, 1))
    expect_ph = pm.Deterministic('expect_ph', tt.exp(tt.dot(x, beta_x) + tt.dot(y, beta_y)))
    expect_hazard = pm.Deterministic('expect_hazard', exposure * tt.outer(expect_ph, lambda0))
    obs_event = pm.Poisson('obs_event', mu=expect_hazard, observed=event)

In [ ]:
with modelA:
    traceA = pm.sample(1000, chains=1)

In [ ]:
list(enumerate(x_di.column_names[1:]))

In [ ]:
# Simulation of counterfactuals

# I use a PPC to sample predictions for
# control and acarbose treated, male mice, at UM
# collected on the same day as the youngest in my real data (779 days).
# First I simulate SCFA concentrations and then I use these
# concentrations as input to a second simulation where I simulate
# death events.

_d = pd.DataFrame({'treatment': ['acarbose', 'control'] * 4,
                   'site': ['UM'] * 4 + ['UT'] * 4,
                   'sex': (['female'] * 2 + ['male'] * 2) * 2})

_x = x_design(_d)

#_x = np.array([[0, 0, 0, 0, 0, 0, 0],
#               [1, 0, 0, 0, 0, 0, 0],
#               [0, 0, 1, 0, 0, 0, 0],
#               [1, 0, 1, 0, 1, 0, 0],
#               [0, 1, 0, 0, 0, 0, 0],
#               [1, 1, 0, 1, 0, 0, 0],
#               [0, 1, 1, 0, 0, 1, 0],
#               [1, 1, 1, 1, 1, 1, 1]], dtype=float)
_y = np.array([[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               [0, 0, 0]], dtype=float)
_exposure = np.ones((8, 56), dtype=float)
_exposure.fill(10)

scfa_sim = np.empty((len(traceA), 8, 3))
event_sim = np.empty((len(traceA), 8, _exposure.shape[-1]))
for i, samp in enumerate(traceA):
    # Acarbose treated mouse
    simA1 = sample_ppc(modelA, [samp], {x: _x, y: _y, exposure: _exposure},
                       vars=[modelA.obs_y], progressbar=False)
    scfa_sim[i] = simA1['obs_y']
    simA2 = sample_ppc(modelA, [samp], {x: _x, y: scfa_sim[i], exposure: _exposure},
                       vars=[modelA.obs_event], progressbar=False)
    event_sim[i] = simA2['obs_event'][0]

In [ ]:
for col_name in ['low', 'mid', 'high']:
    _d[col_name] = np.nan

_d[['low', 'mid', 'high']] = np.percentile(age_at_first(event_sim, interval_bounds), [5, 50, 95], axis=0).T

with temp_assign({x: _x}):
    scfa_expect = (tt.exp(modelA.expect_y)
            .eval({modelA.alpha_0: traceA.alpha_0.mean(0),
                   modelA.alpha: traceA.alpha.mean(0)}))
with temp_assign({x: _x, y: scfa_expect}):
    ph_scfa = modelA.expect_ph.eval({modelA.beta_x: np.zeros((7,1)),
                                     modelA.beta_y: traceA.beta_y.mean(0)})
    ph_design = modelA.expect_ph.eval({modelA.beta_x: traceA.beta_x.mean(0),
                                       modelA.beta_y: np.zeros((3,1))})
    
for i, col_name in enumerate(['propionate', 'butyrate', 'acetate']):
    _d[col_name] = scfa_expect[:,i]
_d['ph_scfa'] = np.log(ph_scfa)
_d['ph_design'] = np.log(ph_design)

_d

In [ ]:
pm.posteriorplot.plot_posterior(traceA, varnames=['beta_y'])
plt.savefig('/Users/bjsmith/Desktop/path_analysis_posterior_y.pdf')

In [ ]:
for treat, (low, mid, high) in zip(y_di.column_names[1:], np.percentile(traceA.beta_y[:,:,0], [2.5, 50, 97.5], 0).T):
    print(f'{treat:<10}\t{low:+0.3f}\t{mid:+0.3f}\t{high:+0.3f}')

In [ ]:
for treat, (low, mid, high) in zip(x_di.column_names[1:], np.percentile(traceA.beta_x[:,:,0], [2.5, 50, 97.5], 0).T):
    print(f'{treat:<45} {low:+0.3}\t{mid:+0.3}\t{high:+0.03}')

In [ ]:
for i, molecule_id in enumerate(y_di.column_names[1:]):
    print(f'**{molecule_id}**')
    for treat, (low, mid, high) in zip(x_di.column_names[1:], np.percentile(traceA.alpha[:,:,i], [2.5, 50, 97.5], 0).T):
        print(f'{treat:<45} {low:+0.3}\t{mid:+0.3}\t{high:+0.03}')
    print()

In [ ]:
pm.traceplot(traceA, varnames=['alpha', 'alpha_0', 'beta_y', 'beta_x'])

In [ ]:
list(enumerate(y_di.column_names[1:]))

In [ ]:
list(enumerate(x_di.column_names[1:]))

In [ ]:
pm.summary(traceA, varnames=['alpha', 'beta_x', 'beta_y'])

In [ ]:
_a, _b = traceA.beta_y[:,0,0], traceA.beta_x[:,0,0]
plt.scatter(_a, _b, s=4)
sp.stats.pearsonr(_a, _b)[0]